In [1]:
# from huggingface_hub import snapshot_download
# snapshot_download(repo_id="stabilityai/stable-diffusion-xl-base-1.0",
#         local_dir="/nobackup3/aryan/instantIR", 
#         local_dir_use_symlinks=False )

In [4]:
# !pip install diffusers opencv-python transformers accelerate
import torch
from PIL import Image

from diffusers import DDPMScheduler
from schedulers.lcm_single_step_scheduler import LCMSingleStepScheduler

from module.ip_adapter.utils import load_adapter_to_pipe
from pipelines.sdxl_instantir import InstantIRPipeline

# suppose you have InstantIR weights under ./models
instantir_path = '/nobackup3/aryan/instantIR/other_wts'

# load pretrained models
pipe = InstantIRPipeline.from_pretrained('/nobackup1/aryan/stable-diffusion-xl-base-1.0', 
                                         torch_dtype=torch.float16, 
                                         local_files_only=True)

# load adapter
load_adapter_to_pipe(
    pipe,
    f"{instantir_path}/adapter.pt",
    image_encoder_or_path = f'/nobackup3/aryan/instantIR/dinov2-large',
)

# load previewer lora
pipe.prepare_previewers(instantir_path)
pipe.scheduler = DDPMScheduler.from_pretrained('/nobackup1/aryan/stable-diffusion-xl-base-1.0', 
                                               subfolder="scheduler", 
                                               local_files_only=True)
lcm_scheduler = LCMSingleStepScheduler.from_config(pipe.scheduler.config)

# load aggregator weights
pretrained_state_dict = torch.load(f"{instantir_path}/aggregator.pt")
pipe.aggregator.load_state_dict(pretrained_state_dict)

# send to GPU and fp16
pipe.to(device='cuda', dtype=torch.float16)
pipe.aggregator.to(device='cuda', dtype=torch.float16)

Loading pipeline components...: 100%|██████████| 7/7 [00:02<00:00,  3.08it/s]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
The config attributes {'interpolation_type': 'linear', 'skip_prk_steps': True, 'use_karras_sigmas': False} were passed to LCMSingleStepScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.


Aggregator(
  (conv_in): Conv2d(4, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (time_proj): Timesteps()
  (time_embedding): TimestepEmbedding(
    (linear_1): Linear(in_features=320, out_features=1280, bias=True)
    (act): SiLU()
    (linear_2): Linear(in_features=1280, out_features=1280, bias=True)
  )
  (add_time_proj): Timesteps()
  (add_embedding): TimestepEmbedding(
    (linear_1): Linear(in_features=2816, out_features=1280, bias=True)
    (act): SiLU()
    (linear_2): Linear(in_features=1280, out_features=1280, bias=True)
  )
  (ref_conv_in): Conv2d(4, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (down_blocks): ModuleList(
    (0): DownBlock2D(
      (resnets): ModuleList(
        (0-1): 2 x ResnetBlock2D(
          (norm1): GroupNorm(32, 320, eps=1e-05, affine=True)
          (conv1): Conv2d(320, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (time_emb_proj): Linear(in_features=1280, out_features=320, bias=True)
          (norm2): G

In [5]:
import random
import math
import os
import numpy as np
import numpy.typing as npt
from typing import List, Dict, cast

def generate_spc_from_gt(img_gt):
    if img_gt is None:
        return None
    img = srgb_to_linearrgb(img_gt / 255.)
    img = emulate_spc(img, 
                      factor=1.0 # Brightness directly proportional to this hparam. 1.0 => scene's natural lighting
                    )
    return img



def srgb_to_linearrgb(img):
    """Performs sRGB to linear RGB color space conversion by reversing gamma
    correction and obtaining values that represent the scene's intensities.

    Args:
        img: npy array or torch tensor

    Returns:
        linear rgb image.
    """
    # https://github.com/blender/blender/blob/master/source/blender/blenlib/intern/math_color.c
    module, img = (torch, img.clone()) if torch.is_tensor(img) else (np, np.copy(img))
    mask = img < 0.04045
    img[mask] = module.clip(img[mask], 0.0, module.inf) / 12.92
    img[~mask] = ((img[~mask] + 0.055) / 1.055) ** 2.4  # type: ignore
    return img


def emulate_spc(
        img, factor: float = 1.0) -> npt.NDArray[np.integer]:
    """Perform bernoulli sampling on linearized RGB frames to yield binary frames.

    Args:
        img (npt.ArrayLike): Linear intensity image to sample binary frame from.
        factor (float, optional): Arbitrary corrective brightness factor. Defaults to 1.0.
        rng (np.random.Generator, optional): Random number generator. Defaults to None.

    Returns:
        Binary single photon frame in [0,1]

        Up --> More PPP
        down --> Low-light

        1    -> 0.3           PPP
        1000 -> 0.3/1000  --> V. Dark
    """
    # Perform bernoulli sampling (equivalent to binomial w/ n=1)
    rng = np.random.default_rng()
    # print("inside rng:", 1.0 - np.exp(-img * factor))
    return rng.binomial(cast(npt.NDArray[np.integer], 1), 1.0 - np.exp(-img * factor))
    
def get_mosaic(img):
        R = img[:, :, 0]
        G = img[:, :, 1]
        B = img[:, :, 2]

        bayer = np.zeros_like(img)

        bayer_pattern_type = random.choice(["RGGB", "GRBG", "BGGR", "GBRG"])

        if bayer_pattern_type == "RGGB":
            # Red
            bayer[0::2, 0::2, 0] = R[0::2, 0::2]
            # Green
            bayer[0::2, 1::2, 1] = G[0::2, 1::2]
            bayer[1::2, 0::2, 1] = G[1::2, 0::2]
            # Blue
            bayer[1::2, 1::2, 2] = B[1::2, 1::2]
        elif bayer_pattern_type == "GRBG":
            # Red
            bayer[0::2, 1::2, 0] = R[0::2, 1::2]
            # Green 
            bayer[0::2, 0::2, 1] = G[0::2, 0::2]
            bayer[1::2, 1::2, 1] = G[1::2, 1::2]
            # Blue
            bayer[1::2, 0::2, 2] = B[1::2, 0::2]
            
        elif bayer_pattern_type == "BGGR":
            # Blue
            bayer[0::2, 0::2, 2] = B[0::2, 0::2]
            # Green
            bayer[0::2, 1::2, 1] = G[0::2, 1::2]
            bayer[1::2, 0::2, 1] = G[1::2, 0::2]
            # Red
            bayer[1::2, 1::2, 0] = R[1::2, 1::2]
        
        else: # GBRG
            # Green
            bayer[0::2, 0::2, 1] = G[0::2, 0::2]
            bayer[1::2, 1::2, 1] = G[1::2, 1::2]
            # Blue
            bayer[0::2, 1::2, 2] = B[0::2, 1::2]
            # Red
            bayer[1::2, 0::2, 0] = R[1::2, 0::2]

        return bayer

In [8]:
from PIL import Image
import os
from tqdm import tqdm

bits = 3
N = 2**bits - 1
color = False

idx = 0
for img_name in tqdm(sorted(os.listdir("/nobackup1/aryan/test_set"))):
    gt = Image.open(f"/nobackup1/aryan/test_set/{img_name}").convert("RGB")
    new_size = (256, 256)
    gt_img = np.array(gt.resize(new_size))
    img_lq_sum = np.zeros_like(gt_img, dtype=np.float32)
    for i in range(N): # 4-bit (2**4 - 1)
        # Bayer:
        if color:
            img_lq_sum = img_lq_sum + get_mosaic(generate_spc_from_gt(gt_img))
        # DeMosaiced:
        else:
            img_lq_sum = img_lq_sum + generate_spc_from_gt(gt_img)
    img_lq = img_lq_sum / (1.0*N)
    img_lq = Image.fromarray((img_lq*255.).astype(np.uint8)) 
    # torch.from_numpy(img_lq).permute(2,0,1).unsqueeze(0).to("cuda")
    # img_lq_tensor = img_lq_tensor.to(torch.float16)
    image = pipe(
        image=img_lq,
        previewer_scheduler=lcm_scheduler,
        prompt=""
    ).images[0]
    image.save(f"/nobackup1/aryan/results/InstantIR/{str(idx).zfill(4)}.png")
    idx += 1

  0%|          | 0/334 [00:00<?, ?it/s]

init latent at 958


  0%|          | 1/334 [00:08<49:54,  8.99s/it]

init latent at 958


  1%|          | 2/334 [00:18<49:53,  9.02s/it]

...

 99%|█████████▉| 332/334 [50:18<00:18,  9.14s/it]

init latent at 958


100%|█████████▉| 333/334 [50:27<00:09,  9.11s/it]

init latent at 958


100%|██████████| 334/334 [50:36<00:00,  9.09s/it]


In [9]:
from PIL import Image
import os
from tqdm import tqdm

bits = 3
N = 2**bits - 1
color = True

idx = 0
for img_name in tqdm(sorted(os.listdir("/nobackup1/aryan/test_set"))):
    gt = Image.open(f"/nobackup1/aryan/test_set/{img_name}").convert("RGB")
    new_size = (256, 256)
    gt_img = np.array(gt.resize(new_size))
    img_lq_sum = np.zeros_like(gt_img, dtype=np.float32)
    for i in range(N): # 4-bit (2**4 - 1)
        # Bayer:
        if color:
            img_lq_sum = img_lq_sum + get_mosaic(generate_spc_from_gt(gt_img))
        # DeMosaiced:
        else:
            img_lq_sum = img_lq_sum + generate_spc_from_gt(gt_img)
    img_lq = img_lq_sum / (1.0*N)
    img_lq = Image.fromarray((img_lq*255.).astype(np.uint8)) 
    # torch.from_numpy(img_lq).permute(2,0,1).unsqueeze(0).to("cuda")
    # img_lq_tensor = img_lq_tensor.to(torch.float16)
    image = pipe(
        image=img_lq,
        previewer_scheduler=lcm_scheduler,
        prompt=""
    ).images[0]
    image.save(f"/nobackup1/aryan/results/InstantIR/color_{str(idx).zfill(4)}.png")
    idx += 1

  0%|          | 0/334 [00:00<?, ?it/s]

init latent at 958


  0%|          | 1/334 [00:09<50:27,  9.09s/it]

init latent at 958


  1%|          | 2/334 [00:18<50:00,  9.04s/it]

...

 99%|█████████▉| 332/334 [50:19<00:18,  9.13s/it]

init latent at 958


100%|█████████▉| 333/334 [50:29<00:09,  9.12s/it]

init latent at 958


100%|██████████| 334/334 [50:38<00:00,  9.10s/it]


In [20]:
import piq
def compute_full_reference_metrics(gt_img, out_img):
    psnr = piq.psnr(out_img, gt_img, data_range=1., reduction='none')
    ssim = piq.ssim(out_img, gt_img, data_range=1.) 
    lpips = piq.LPIPS(reduction='none')(out_img, gt_img)
    return psnr.item(), ssim.item(), lpips.item()


import pyiqa
# ManIQA DeQA MUSIQ ClipIQA
maniqaMODEL = pyiqa.create_metric('maniqa', device=torch.device("cuda"))
clipiqaMODEL = pyiqa.create_metric('clipiqa', device=torch.device("cuda"))
musiqMODEL = pyiqa.create_metric('musiq', device=torch.device("cuda"))
# from DeQAScore.src import Scorer
def compute_no_reference_metrics(out_img):
    _, _, h, w = out_img.shape
    top = (h - 224) // 2
    left = (w - 224) // 2
    out_img = out_img[:, :, top:top+224, left:left+224]

    maniqa_score = maniqaMODEL(out_img).item()
    clipiqa_score = clipiqaMODEL(out_img).item()
    musiq_score = musiqMODEL(out_img).item()

    return maniqa_score, clipiqa_score, musiq_score #, deqa_score

def calcMetrics(gt, out):
    psnr, ssim, lpips, manIQA, clipIQA, musiq = None, None, None, None, None, None
    psnr, ssim, lpips = compute_full_reference_metrics(gt, out)
    manIQA, clipIQA, musiq = compute_no_reference_metrics(out)
    return psnr, ssim, lpips, manIQA, clipIQA, musiq


Loading pretrained model MANIQA from /tmp/21315/cache/torch/hub/pyiqa/ckpt_koniq10k.pt
Loading pretrained model MUSIQ from /tmp/21315/cache/torch/hub/pyiqa/musiq_koniq_ckpt-e95806b9.pth


In [21]:
from torchvision import transforms

In [23]:
from PIL import Image
import os
from tqdm import tqdm

bits = 3
N = 2**bits - 1
color = False

idx = 0
psnr_list, ssim_list, lpips_list, manIQA_list, clipIQA_list, musiq_list = [], [], [], [], [], []
for img_name in tqdm(sorted(os.listdir("/nobackup1/aryan/test_set"))):
    gt = Image.open(f"/nobackup1/aryan/test_set/{img_name}").convert("RGB")
    new_size = (256, 256)
    gt_img = np.array(gt.resize(new_size))
    img_lq_sum = np.zeros_like(gt_img, dtype=np.float32)
    for i in range(N): # 4-bit (2**4 - 1)
        # Bayer:
        if color:
            img_lq_sum = img_lq_sum + get_mosaic(generate_spc_from_gt(gt_img))
        # DeMosaiced:
        else:
            img_lq_sum = img_lq_sum + generate_spc_from_gt(gt_img)
    img_lq = img_lq_sum / (1.0*N)
    img_lq = Image.fromarray((img_lq*255.).astype(np.uint8)) 
    # torch.from_numpy(img_lq).permute(2,0,1).unsqueeze(0).to("cuda")
    # img_lq_tensor = img_lq_tensor.to(torch.float16)
    out = Image.open(f"/nobackup1/aryan/results/InstantIR/{str(idx).zfill(4)}.png").convert("RGB")
    gt_tensor = transforms.ToTensor()(gt_img).unsqueeze_(0)
    out_tensor = transforms.ToTensor()(out).unsqueeze_(0)
    psnr, ssim, lpips, manIQA, clipIQA, musiq = calcMetrics(gt_tensor, out_tensor)
    psnr_list.append(psnr)
    ssim_list.append(ssim)
    lpips_list.append(lpips)
    manIQA_list.append(manIQA)
    clipIQA_list.append(clipIQA)
    musiq_list.append(musiq)
    idx += 1

print("PSNR:", np.mean(psnr_list))
print("SSIM:", np.mean(ssim_list))
print("LPIPS:", np.mean(lpips_list))
print("----------------------------")
print("ManIQA:", np.mean(manIQA_list))
print("ClipIQA:", np.mean(clipIQA_list))
print("MUSIQ:", np.mean(musiq_list))

  0%|          | 0/334 [00:00<?, ?it/s]/nobackup/aryan/.conda/envs/sd35/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/nobackup/aryan/.conda/envs/sd35/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 334/334 [04:40<00:00,  1.19it/s]

PSNR: 10.7876510463075
SSIM: 0.17841990092423504
LPIPS: 0.6517673761187913
----------------------------
ManIQA: 0.18767385004077128
ClipIQA: 0.34627629687150796
MUSIQ: 36.651796983387655


In [24]:
from PIL import Image
import os
from tqdm import tqdm

bits = 3
N = 2**bits - 1
color = True

idx = 0
psnr_list, ssim_list, lpips_list, manIQA_list, clipIQA_list, musiq_list = [], [], [], [], [], []
for img_name in tqdm(sorted(os.listdir("/nobackup1/aryan/test_set"))):
    gt = Image.open(f"/nobackup1/aryan/test_set/{img_name}").convert("RGB")
    new_size = (256, 256)
    gt_img = np.array(gt.resize(new_size))
    img_lq_sum = np.zeros_like(gt_img, dtype=np.float32)
    for i in range(N): # 4-bit (2**4 - 1)
        # Bayer:
        if color:
            img_lq_sum = img_lq_sum + get_mosaic(generate_spc_from_gt(gt_img))
        # DeMosaiced:
        else:
            img_lq_sum = img_lq_sum + generate_spc_from_gt(gt_img)
    img_lq = img_lq_sum / (1.0*N)
    img_lq = Image.fromarray((img_lq*255.).astype(np.uint8)) 
    # torch.from_numpy(img_lq).permute(2,0,1).unsqueeze(0).to("cuda")
    # img_lq_tensor = img_lq_tensor.to(torch.float16)
    out = Image.open(f"/nobackup1/aryan/results/InstantIR/color_{str(idx).zfill(4)}.png").convert("RGB")
    gt_tensor = transforms.ToTensor()(gt_img).unsqueeze_(0)
    out_tensor = transforms.ToTensor()(out).unsqueeze_(0)
    psnr, ssim, lpips, manIQA, clipIQA, musiq = calcMetrics(gt_tensor, out_tensor)
    psnr_list.append(psnr)
    ssim_list.append(ssim)
    lpips_list.append(lpips)
    manIQA_list.append(manIQA)
    clipIQA_list.append(clipIQA)
    musiq_list.append(musiq)
    idx += 1

print("PSNR:", np.mean(psnr_list))
print("SSIM:", np.mean(ssim_list))
print("LPIPS:", np.mean(lpips_list))
print("----------------------------")
print("ManIQA:", np.mean(manIQA_list))
print("ClipIQA:", np.mean(clipIQA_list))
print("MUSIQ:", np.mean(musiq_list))

  0%|          | 0/334 [00:00<?, ?it/s]/nobackup/aryan/.conda/envs/sd35/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/nobackup/aryan/.conda/envs/sd35/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 334/334 [04:38<00:00,  1.20it/s]

PSNR: 7.928232275797222
SSIM: 0.10106084266019438
LPIPS: 0.7364063048791029
----------------------------
ManIQA: 0.19743301469029603
ClipIQA: 0.3586033666562177
MUSIQ: 32.21109190101395
